#### Import all the necessary modules.

In [1]:
import matplotlib.pyplot as plt
import spacy
# from spacy.training.example import Example
import random
from tqdm import tqdm

import pandas as pd
import boto3
from sqlalchemy import create_engine
import time

import seaborn as sns
import matplotlib.pyplot as plt

from requests.exceptions import ReadTimeout
import time
from tqdm import tqdm

from datetime import date, timedelta
import numpy as np
import re
import json
import string
import pysftp
from botocore.exceptions import ClientError
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None

today = date.today()
yesterday = today - timedelta(days=1)
start_date = str(yesterday)


from sklearn.model_selection import train_test_split

#### For Running Query using Redash Tables.

In [2]:
def get_engine_client_dashboard():
    import boto3
    from sqlalchemy import create_engine
    ssm_keys = ['prod_client_dashboard_postgres_host', 'prod_client_dashboard_postgres_name', 'prod_client_dashboard_postgres_ro_user', 'prod_client_dashboard_postgres_ro_pwd']
    ssm_parameters = boto3.client('ssm', region_name='ap-south-1').get_parameters(Names=ssm_keys, WithDecryption=True)['Parameters']
    ssm_secrets = dict()
    for ssm_parameter in ssm_parameters:
        ssm_secrets[ssm_parameter['Name']] = ssm_parameter['Value']
    username = ssm_secrets.get('prod_client_dashboard_postgres_ro_user')
    password = ssm_secrets.get('prod_client_dashboard_postgres_ro_pwd')
    hostname = ssm_secrets.get('prod_client_dashboard_postgres_host')
    databasename = ssm_secrets.get('prod_client_dashboard_postgres_name')
    engine_link = 'postgresql+psycopg2://{}:{}@{}/{}'.format(username, password, hostname, databasename)
    engine = create_engine(engine_link)
    return engine
engine_client_dashboard = get_engine_client_dashboard()

In [3]:
def get_aws_creds(database):
    """Get Redshift credentials from AWS Systems Manager (SSM)"""
    ssm_keys = ['prod_redshift_host', 'prod_redshift_ro_password', 'prod_redshift_port', 'prod_redshift_ro_user',
                'lending_readonly_host', 'lending_readonly_pass', 'lending_readonly_port', 'lending_readonly_user',
                database]
    ssm_parameters = boto3.client('ssm', region_name='ap-south-1').get_parameters(Names=ssm_keys, WithDecryption=True)['Parameters']
    ssm_secrets = dict()
    for ssm_parameter in ssm_parameters:
        ssm_secrets[ssm_parameter['Name']] = ssm_parameter['Value']
    return ssm_secrets
def get_engine(database):
    ssm_secrets = get_aws_creds(database)
    if database == 'prod_redshift_name':
        username = ssm_secrets.get('prod_redshift_ro_user')
        password = ssm_secrets.get('prod_redshift_ro_password')
        hostname = ssm_secrets.get('prod_redshift_host')
        port_number = ssm_secrets.get('prod_redshift_port')
        databasename = ssm_secrets.get(database)
        engine_link = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(username, password, hostname, port_number, databasename)
        engine = create_engine(engine_link)
    elif database == 'lending_readonly_dbname':
        username = ssm_secrets.get('lending_readonly_user')
        password = ssm_secrets.get('lending_readonly_pass')
        hostname = ssm_secrets.get('lending_readonly_host')
        port_number = ssm_secrets.get('lending_readonly_port')
        databasename = ssm_secrets.get(database)
        engine_link = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(username, password, hostname, port_number, databasename)
        engine = create_engine(engine_link)
    elif database == 'UNIFIED_READ_ONLY_DB':
        username = ssm_secrets.get('UNIFIED_READ_ONLY_USER')
        password = ssm_secrets.get('UNIFIED_READ_ONLY_PASSWORD')
        hostname = ssm_secrets.get('UNIFIED_READ_ONLY_HOST')
        port_number = ssm_secrets.get('UNIFIED_READ_ONLY_PORT')
        databasename = ssm_secrets.get(database)
        engine_link = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(username, password, hostname, port_number, databasename)
        engine = create_engine(engine_link)
    return engine


def get_engine_unified(database):
    ssm_keys = ['UNIFIED_READ_ONLY_USER','UNIFIED_READ_ONLY_PASSWORD','UNIFIED_READ_ONLY_HOST','UNIFIED_READ_ONLY_PORT',
                database]
    ssm_parameters = boto3.client('ssm', region_name='ap-south-1').get_parameters(Names=ssm_keys, WithDecryption=True)['Parameters']
    ssm_secrets = dict()
    for ssm_parameter in ssm_parameters:
        ssm_secrets[ssm_parameter['Name']] = ssm_parameter['Value']
    username = ssm_secrets.get('UNIFIED_READ_ONLY_USER')
    password = ssm_secrets.get('UNIFIED_READ_ONLY_PASSWORD')
    hostname = ssm_secrets.get('UNIFIED_READ_ONLY_HOST')
    port_number = ssm_secrets.get('UNIFIED_READ_ONLY_PORT')
    databasename = ssm_secrets.get(database)
    engine_link = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(username, password, hostname, port_number, databasename)
    engine = create_engine(engine_link)
    return engine

def get_engine_banking(database):
    ssm_keys = ['PORTAL_READONLY_USER','PORTAL_READONLY_HOST','PORTAL_READONLY_PASSWORD']
    ssm_parameters = boto3.client('ssm', region_name='ap-south-1').get_parameters(Names=ssm_keys, WithDecryption=True)['Parameters']
    ssm_secrets = dict()
    for ssm_parameter in ssm_parameters:
        ssm_secrets[ssm_parameter['Name']] = ssm_parameter['Value']
    username = ssm_secrets.get('PORTAL_READONLY_USER')
    password = ssm_secrets.get('PORTAL_READONLY_PASSWORD')
    hostname = ssm_secrets.get('PORTAL_READONLY_HOST')
    databasename = database
    engine_link = 'postgresql+psycopg2://{}:{}@{}/{}'.format(username, password, hostname, databasename)
    engine = create_engine(engine_link)
    return engine


engine_dw2 = get_engine('prod_redshift_name')
engine_lending = get_engine('lending_readonly_dbname')
engine_unified = get_engine_unified('UNIFIED_READ_ONLY_DB')
engine_bank_connect = get_engine_banking('finbox_dashboard')
engine_client_dashboard = get_engine_client_dashboard()

In [4]:
# !pip show spacy

In [5]:
# working_df_orig = pd.read_sql("""with CTE1 as
# (SELECT distinct smsbody, smstype, row_number() over(partition by smstype order by rand()) as rn1
# from extractedlive_v2 
# where dt = '2024-06-18')

# select smsbody, smstype from CTE1 where rn1<= 50000;""", engine_dw2)

#### For Running Query using Athena Tables.

In [6]:
# aws setup
import pandas as pd
import boto3
import time
import io
import botocore

In [7]:
database = 'redshiftdump'
# s3_input = 's3://' + self.bucket + '/my_folder_input'
bucket_name = 'aws-athena-query-results-661900564209-ap-south-1'
folder_name = 'Featurisation/'
s3_output =  's3://'+bucket_name+ '/' + folder_name
region_name = 'ap-south-1'
# aws_access_key_id = "xyz"
# aws_secret_access_key = "xyz"

def load_conf(q):
    client = boto3.client('athena',
                      region_name = region_name
                      #     ,
                      # aws_access_key_id = aws_access_key_id,
                      # aws_secret_access_key= aws_secret_access_key
    )
    response = client.start_query_execution(
        QueryString = q,
            QueryExecutionContext={
            'Database': database
            },
            ResultConfiguration={
            'OutputLocation': s3_output,
            },
            WorkGroup='restricted'
 
    )
    filename = response['QueryExecutionId']
    print('Execution ID: ' + response['QueryExecutionId'])
    return filename
 
def get_id_data(id):
 
    client = boto3.client('athena',
                          region_name=region_name
                          # ,
                          # aws_access_key_id=aws_access_key_id,
                          # aws_secret_access_key=aws_secret_access_key
                          )
 
 
    try_count = 0
    state = 'RUNNING'
    while (state in ['RUNNING','QUEUED']) :
        try_count +=1
        try:
            response = client.batch_get_query_execution(QueryExecutionIds=[id])
            state = response['QueryExecutions'][0]['Status']['State']
        except:
            pass
 
    if state =='SUCCEEDED':
        data  = obtain_data(id)
        return data
 
    elif state =='FAILED':
        raise  ValueError(str(response['QueryExecutions'][0]['Status']['StateChangeReason']))
 
    elif state =='CANCELLED':
        raise ValueError('\nQuery Cancelled.')
    else:
        raise ValueError('\nSome unknown Error occured.')
 
 
def obtain_data(execution_id, max_try_count =40,initial_sleep = 3,loop_sleep_time = 5, format = '.csv'):
    time.sleep(initial_sleep)
    try_count = 0
    while(try_count<max_try_count):
        try:
            resource = boto3.resource('s3',
                                  region_name = region_name
                                  #     ,
                                  # aws_access_key_id = aws_access_key_id,
                                  # aws_secret_access_key= aws_secret_access_key
                                      )
            response = resource \
            .Bucket(bucket_name) \
            .Object(key= folder_name + execution_id + format) \
            .get()
            return pd.read_csv(io.BytesIO(response['Body'].read()), encoding='utf8')
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                try_count += 1
                time.sleep(loop_sleep_time)
            else:
                return "No data found in ",max_try_count, " tries.\n Error: ",e
 
 
def run_athena_query(query):
    id = load_conf(query)
 
    query_result = get_id_data(id)
    return query_result
 
def get_id_status(id):
    client = boto3.client('athena',
                          region_name=region_name
                          # ,
                          # aws_access_key_id=aws_access_key_id,
                          # aws_secret_access_key=aws_secret_access_key
                          )
    response = client.batch_get_query_execution(QueryExecutionIds=[id])
    state = response['QueryExecutions'][0]['Status']['State']
    return state

In [8]:
# idn_users_df = run_athena_query("""select subusername, hash, sdk_timestamp, client_id, user_hash, dt, sim_carrier_name_1, sim_carrier_name_2,
# sim_country_iso_1, sim_country_iso_2, sim_data_roaming_1, sim_data_roaming_2, sim_slot_index_1, sim_slot_index_2,
# sim_number_1, sim_number_2, sim_id, display_country
# from public_device_data
# where client_id = '1' and (lower(sim_country_iso_1) = 'id' or lower(sim_country_iso_2) = 'id');""")

#### Check versions.

In [9]:
spacy.__version__

'3.7.5'

In [10]:
!python --version

Python 3.7.6


#### For running query using click house tables.

In [11]:
# import time
# import pandas as pd
# import boto3
# import clickhouse_connect

# def get_ssm_parameters(parameters: list):
#     ssm_parameters = boto3.client('ssm', region_name='ap-south-1').get_parameters(Names=parameters, WithDecryption=True)['Parameters']
#     ssm_secrets = dict()
#     for ssm_parameter in ssm_parameters:
#         ssm_secrets[ssm_parameter['Name']] = ssm_parameter['Value']
#     return ssm_secrets

# class CHConnection:
#     def __init__(self, host: str, database: str, user: str, password: str, port: int=9000):
#         self.ch_client = clickhouse_connect.get_client(host=host,port=port,user=user,password=password,database=database)

#     def execute_query(self, query: str) -> list:
#         result = self.ch_client.query(query)
#         columns = result.column_names
#         data = [dict(zip(columns, row)) for row in result.result_rows]
#         return data

# def create_ch_engine(product: str) -> CHConnection:
#     parameter_prefix = product.upper()
#     db_param = f'{parameter_prefix}_CLICKHOUSE_DB'
#     host_param = f'{parameter_prefix}_CLICKHOUSE_HOST'
#     password_param = f'{parameter_prefix}_CLICKHOUSE_PASSWORD'
#     port_param = f'{parameter_prefix}_CLICKHOUSE_PORT'
#     user_param = f'{parameter_prefix}_CLICKHOUSE_USER'

#     parameters = [db_param, host_param, password_param, port_param, user_param]
#     credentials = get_ssm_parameters(parameters=parameters)

#     ch_connector = CHConnection(host=credentials[host_param], database=credentials[db_param],user=credentials[user_param],
#         password=credentials[password_param],port=int(credentials[port_param]))
#     return ch_connector

# def execute_ch_query(product, query):
#     engine = create_ch_engine(product=product)
#     df = pd.DataFrame()
#     i=0
#     while((i<15) and len(df)==0):
#         try:
#             if (i>0):
#                 print('Fetching...')
#             df = pd.DataFrame(engine.execute_query(query))
#             i = i+1            
#         except Exception as e:
#             i = i+1
#             print(e)
#             print('--- CH Data Fetching failed {} time---'.format(i))
#             time.sleep(1)
#     return df

# dc_ch_engine = create_ch_engine(product='dc_mirrored')

# # dc_df = execute_ch_query(product='dc_mirrored', query='sdfvsdv')
# # lending_df = execute_ch_query(product='lending', query='sdfvsdv')
# sentinel_df = execute_ch_query(product='sentinel', query='sdfvsdv')

In [12]:
# !pip install clickhouse-connect

In [13]:
# sms_query = '''select * from device_connect_mirror.raw_sms limit 100 '''

In [14]:
# sms_data = pd.DataFrame(dc_ch_engine.execute_query(sms_query))

In [15]:
# sms`_data.head()

#### Prepare test data for testing the trained model.

In [16]:
templates_used_for_training = tuple(['74d4e4dcffd6f0838739568ee0f070d7',
                    '8329648f7951ad7c16bf24daf43d14f0',
                    '73f468b47e7570a476884f9878b61d24',
                    '8e758449c9752f4ccb419cf053f16d37',
                    '8b2e91c03550330bf60e9fbd50c0553b'])

In [17]:
# Test sample

In [18]:
# test_data_df = run_athena_query(f"""with CTE1 as
# (SELECT smsbody, templatehash, hash, smstype, amount, balance, pan, date,row_number() over(partition by templatehash order by rand()) as rn
# from extractedlive_v2
# where dt between '2024-05-01' and '2024-05-25'
# and smstype in ('credit-transaction', 'debit-transaction')
# and accounttype in ('bank')
# and templatehash not in {templates_used_for_training}
# limit 100000
# )

# select smsbody, templatehash, hash, smstype, amount, balance, pan, date
# from CTE1 where rn<= 100;""")


test_data_df = run_athena_query(f"""with CTE1 as
(SELECT smsbody, templatehash, hash, smstype, amount, balance, pan, date,row_number() over(partition by templatehash order by rand()) as rn
from extractedlive_v2
where dt = '2024-05-01'
and smstype in ('credit-transaction', 'debit-transaction')
limit 1000000
)

select smsbody, templatehash, hash, smstype, amount, balance, pan, date
from CTE1 where rn<= 1000;""")

Execution ID: 416e8e41-0918-49a9-970b-4555eaadd806


In [19]:
test_data_df.shape, test_data_df['hash'].nunique(), test_data_df['templatehash'].nunique()

((194606, 8), 182332, 363)

In [20]:
test_data_df.head(2)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date
0,DBGB: Ac XXXXXXXX003338 Debited with Rs.158.00...,4bb988f625ed5311465b87a3b42984cd,1a2329fdadea93a4e0f929a321fd9e54,debit-transaction,158.00,5111.00,XXXXXXXX003338,02-11-2022
1,"DBGB: Ac XXXXXXXX509006 Debited with Rs.17.70,...",4bb988f625ed5311465b87a3b42984cd,02d29bf975b65db3c0122bd302b6b780,debit-transaction,17.69,152.59,XXXXXXXX509006,27-06-2024


In [21]:
test_data_df = test_data_df.drop_duplicates(subset = ['hash'], ignore_index = True)

In [22]:
test_data_df.shape, test_data_df['templatehash'].nunique(), test_data_df['hash'].nunique()

((182332, 8), 363, 182332)

In [23]:
test_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182332 entries, 0 to 182331
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   smsbody       182332 non-null  object 
 1   templatehash  182332 non-null  object 
 2   hash          182332 non-null  object 
 3   smstype       182332 non-null  object 
 4   amount        182332 non-null  float64
 5   balance       182332 non-null  float64
 6   pan           182332 non-null  object 
 7   date          182332 non-null  object 
dtypes: float64(2), object(6)
memory usage: 11.1+ MB


In [24]:
test_data_df['templatehash'].value_counts()

1e3fd94860b6ed8bb42897105d3d3fdd    1000
1a07d5aa88a641e3658516653bdf0cff    1000
21210289b440b1fdd65b8830657e46aa    1000
73e1909fcee893118b581bec704b9621     998
9a2915937ca0fca9c13730f3caf247f9     998
                                    ... 
2552be17ee2b5d23a129cc2521d4da76       1
83be6098ed031a23d1bba02665f43756       1
2520ceb8cbfb900d05bec17b5e5da9e5       1
1945d04ff4350a1da7787dfcf138bdcc       1
2c0c30c7eda8b79f1893027696fd6ef8       1
Name: templatehash, Length: 363, dtype: int64

In [25]:
# test_data = test_data_df[test_data_df['templatehash'] == 'f8f6bb17cc6195c7dbe07dd295e06c74'].copy()
test_data = test_data_df.copy()

In [26]:
test_data.head()

,smsbody,templatehash,hash,smstype,amount,balance,pan,date
0,DBGB: Ac XXXXXXXX003338 Debited with Rs.158.00...,4bb988f625ed5311465b87a3b42984cd,1a2329fdadea93a4e0f929a321fd9e54,debit-transaction,158.00,5111.00,XXXXXXXX003338,02-11-2022
1,"DBGB: Ac XXXXXXXX509006 Debited with Rs.17.70,...",4bb988f625ed5311465b87a3b42984cd,02d29bf975b65db3c0122bd302b6b780,debit-transaction,17.69,152.59,XXXXXXXX509006,27-06-2024
2,DBGB: Ac XXXXXXXX059544 Debited with Rs.4000.0...,4bb988f625ed5311465b87a3b42984cd,b531316c9d60e0991dd956b708a36ea3,debit-transaction,4000.00,1128.72,XXXXXXXX059544,02-02-2023
3,DBGB: Ac XXXXXXXX001873 Debited with Rs.2000.0...,4bb988f625ed5311465b87a3b42984cd,8c8d613712f4aa88c5e668f1fdaf84c2,debit-transaction,2000.00,1670.00,XXXXXXXX001873,08-03-2024
4,DBGB: Ac XXXXXXXX003338 Debited with Rs.604.00...,4bb988f625ed5311465b87a3b42984cd,783242c5c17bb90260433db8c046a6fe,debit-transaction,604.00,4579.00,XXXXXXXX003338,11-11-2022


In [27]:
test_data.shape

(182332, 8)

### Preprocessing the text:

#### Strip the values to remove leading and trailing spaces.

In [28]:
def strip_text(value):
    if pd.isna(value):
        return np.nan  # Keep NaN values as NaN
    # Strip spaces and convert to string
    value_str = str(value).strip()
    return value_str

In [29]:
test_data['pan'] = test_data['pan'].apply(lambda x : strip_text(x))
test_data['amount'] = test_data['amount'].apply(lambda x : strip_text(x))
test_data['date'] = test_data['date'].apply(lambda x : strip_text(x))
test_data['balance'] = test_data['balance'].apply(lambda x : strip_text(x))

In [30]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182332 entries, 0 to 182331
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   smsbody       182332 non-null  object
 1   templatehash  182332 non-null  object
 2   hash          182332 non-null  object
 3   smstype       182332 non-null  object
 4   amount        182332 non-null  object
 5   balance       182332 non-null  object
 6   pan           182332 non-null  object
 7   date          182332 non-null  object
dtypes: object(8)
memory usage: 11.1+ MB


In [31]:
pd.set_option('display.max_colwidth', None)
test_data.head(2)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date
0,"DBGB: Ac XXXXXXXX003338 Debited with Rs.158.00,02-11-2022 17:36:11. Aval Bal Rs.5111.00 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,1a2329fdadea93a4e0f929a321fd9e54,debit-transaction,158.0,5111.0,XXXXXXXX003338,02-11-2022
1,"DBGB: Ac XXXXXXXX509006 Debited with Rs.17.70,27-06-2024 20:30:29. Aval Bal Rs.152.60 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,02d29bf975b65db3c0122bd302b6b780,debit-transaction,17.69,152.59,XXXXXXXX509006,27-06-2024


In [32]:
test_data.tail(2)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date
182330,"ICICI Bank Acc XX670 debited Rs. 10,000.00 on 28-APR-24 NFS*CASH WDL*. Avb Bal Rs. 46,406.85. To dispute Call 18002662 or SMS BLOCK 670 to 9215676766 .",f4b0f336ff0669fd8fdedad3f9b0ea82,6cfb289b5e85f3f1e8e656408aeb2377,debit-transaction,10000.0,46406.84,XX670,28-APR-24
182331,"ICICI Bank Acc XX291 debited Rs. 1,500.00 on 01-Jan-24 NFS*CASH WDL*. Avb Bal Rs. 171.74. To dispute Call 18002662 or SMS BLOCK 291 to 9215676766 .",f4b0f336ff0669fd8fdedad3f9b0ea82,c85e16dc0536d16138a6b413f143cc12,debit-transaction,1500.0,171.74,XX291,01-Jan-24


####  Add space before and after currency symbol.

In [33]:
def preprocess_sms1(text):
    # Add a space after 'Rs.' or 'INR' followed by numbers
    pattern = r'(Rs\.|Rs:|INR\.|INR:|Rs|INR|I@NR|:|\(|\))'
    # Substituting by adding a space after the decimal number
    processed_sms = re.sub(pattern, r' \1 ', text)
    return processed_sms

In [34]:
test_data['smsbody_1'] = test_data['smsbody'].apply(lambda x : preprocess_sms1(x))

In [35]:
# test_data.head(3)

#### Handle 'A/c no. XX109016-12-23 00 : 06 : 51,'

In [36]:
# text5 = 'Debit INR 20000.00 Acc num XX*10901232416-12-24 00 : 06 : 51 ISTIMPS/P2A/335000170409/AbhiBal INR 60316.47 SMS BLOCKALL Cust ID to 919951860002, if not you - Axis Bank'

In [37]:
def preprocess_sms2(text):
    # Add a space after 'Rs.' or 'INR' followed by numbers
    pattern = r'(?i)(account|acc\.|ac\.|acct\.|a/c\.|account\.|acc|ac|acct|a/c)(\s*)(no\.:|number\.:|num\.:|no\.|number\.|numbr\.|num\.|no:|number:|num:|No|number|numbr|num)(\s*)([X\d\*]+)(\d{2}-\d{2}-\d{2})'
    # Substituting by adding a space after the decimal number
    processed_sms = re.sub(pattern, r'\1\2\3\4 \5 \6', text)
    return processed_sms

In [38]:
test_data['smsbody_2'] = test_data['smsbody_1'].apply(lambda x : preprocess_sms2(x))

#### Handle 'INR  9800013-07-24 16 : 37 : 57RAZ*GREENPAAvl'

In [39]:
def preprocess_sms3(text):
    # Add a space after 'Rs.' or 'INR' followed by numbers
    pattern = r'(?i)(Rs\.|Rs:|INR\.|INR:|Rs|INR|I@NR)(\s+)([\d\.\,]+)(\d{2}-\d{2}-\d{2})'
    # Substituting by adding a space after the decimal number
    processed_sms = re.sub(pattern, r'\1\2\3 \4', text)
    return processed_sms

In [40]:
test_data['smsbody_3'] = test_data['smsbody_2'].apply(lambda x : preprocess_sms3(x))

#### Handle 'Rs. 10,000.00,28-05-2024 20 : 32 : 19'

In [41]:
# text3 = 'Chq.344115, Rs. 10,00.25 , fvg. BISHAN DAS AND SONS debited from A/C XXXXXXXX00000359 , thru clg.-avail Bal Rs. 5901.37.Helpline 18001802222-PNB INR 4399.-Kotak INR 44,918.52.-SBI Rs. 6024.24- Federal A/c XX9869 debited with Rs. 10,000.00,28-05-2024 20 : 32 : 19,RRN414920031702 thru card XX0742.'

In [42]:
def preprocess_sms4(text):
    # Add a space after 'Rs.' or 'INR' followed by numbers
    pattern = r'(?i)(Rs\.|Rs:|INR\.|INR:|Rs|INR|I@NR)(\s+)([\d\.\,]+)(\,)(\d{2}-\d{2}-\d{2,4})'
    # Substituting by adding a space after the decimal number
    processed_sms = re.sub(pattern, r'\1\2\3 \4 \5', text)
    return processed_sms

In [43]:
test_data['smsbody_4'] = test_data['smsbody_3'].apply(lambda x : preprocess_sms4(x))

#### handle 'Rs. 5901.37.Helpline'

In [44]:
# text2 = 'Chq.344115, Rs. 10,00.25 , fvg. BISHAN DAS AND SONS debited from A/C XXXXXXXX00000359 , thru clg.-avail Bal Rs. 5901.37.Helpline 18001802222-PNB INR 4399.-Kotak INR 44,918.52.-SBI Rs. 6024.24- Federal'

In [45]:
def preprocess_sms5(text):
    # Add a space after 'Rs.' or 'INR' followed by numbers
    pattern = r'(?i)(Rs\.|Rs:|INR\.|INR:|Rs|INR|I@NR)(\s+)([\d\.\,]+)(\.|\,)([A-z]|-)'
    # Substituting by adding a space after the decimal number
    processed_sms = re.sub(pattern, r'\1\2\3 \4 \5', text)
    return processed_sms

In [46]:
test_data['smsbody_5'] = test_data['smsbody_4'].apply(lambda x : preprocess_sms5(x))

#### Handle 'Rs. 216.65CR.'

In [47]:
def preprocess_sms6(text):
    # Add a space after 'Rs.' or 'INR' followed by numbers
    pattern = r'(?i)(Rs\.|Rs:|INR\.|INR:|Rs|INR|I@NR)(\s+)([\d\.\,]+)([A-z])'
    # Substituting by adding a space after the decimal number
    processed_sms = re.sub(pattern, r'\1\2\3 \4', text)
    return processed_sms

In [48]:
test_data['smsbody_6'] = test_data['smsbody_5'].apply(lambda x : preprocess_sms6(x))

####  Split the text on space and then join to remove leading and trailing spaces.

In [49]:
test_data['final_smsbody'] = test_data['smsbody_6'].apply(lambda x: ' '.join(x.split()))

In [50]:
test_data.head(3)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,smsbody_3,smsbody_4,smsbody_5,smsbody_6,final_smsbody
0,"DBGB: Ac XXXXXXXX003338 Debited with Rs.158.00,02-11-2022 17:36:11. Aval Bal Rs.5111.00 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,1a2329fdadea93a4e0f929a321fd9e54,debit-transaction,158.0,5111.0,XXXXXXXX003338,02-11-2022,"DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777"
1,"DBGB: Ac XXXXXXXX509006 Debited with Rs.17.70,27-06-2024 20:30:29. Aval Bal Rs.152.60 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,02d29bf975b65db3c0122bd302b6b780,debit-transaction,17.69,152.59,XXXXXXXX509006,27-06-2024,"DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70,27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70,27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777"
2,"DBGB: Ac XXXXXXXX059544 Debited with Rs.4000.00,02-02-2023 12:08:39. Aval Bal Rs.1128.72 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,b531316c9d60e0991dd956b708a36ea3,debit-transaction,4000.0,1128.72,XXXXXXXX059544,02-02-2023,"DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00,02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00,02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00, 02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00, 02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00, 02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00, 02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00, 02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777"


In [51]:
test_data.shape, test_data['templatehash'].nunique()

((182332, 15), 363)



#### 2. Function to remove extra spaces

In [52]:
# test_data['smsbody_1'] = test_data['smsbody'].apply(lambda x: ' '.join(x.split()))

In [53]:
# test_data.shape, test_data['templatehash'].nunique()

In [54]:
# test_data.head(2)

#### 3. Impute space after amount / balance decimal values if there is no space.

In [55]:
# import re

# # Example SMS
# sms = """Your Ac No. 023/XXXXX249 has been credited by INR.9856.07On 05-JAN-2024 by Transfer rs.10.Info- NEFT PHONEPE PRIVATE LIMITED PAYMENT AG 922020004688715 UTIB0001506 AX. Your net available balance is Rs. 195918.08GPPSB."""

In [56]:
# # # Pattern to match 'Rs.' or 'INR' followed by a number with decimal points
# # # Ensure it only matches if there's no space right after the number

# def preprocess_sms1(text):
#     # Add a space after 'Rs.' or 'INR' followed by numbers
#     pattern = r'(Rs\.|INR\.|Rs|INR)(\s*)(\d+\.\d{2})(?!\s)'
#     # Substituting by adding a space after the decimal number
#     processed_sms = re.sub(pattern, r'\1\2\3 ', text)
#     return processed_sms

# # import re

# # def preprocess_sms1(text):
# #     # Pattern to match amounts preceded by Rs. or INR with or without space, and ensure space follows
# #     pattern = r'((Rs\.|INR\.|Rs|INR)\s*(\d+\.\d{2})(?!\s))|((Rs\.|INR\.|Rs|INR)\s*(\d+)(?!\s))'  
# #     # Substituting by adding a space after the matched amounts
# #     processed_sms = re.sub(pattern, lambda match: match.group(0) + ' ', text)
# #     return processed_sms


In [57]:
# # Process the SMS
# processed_sms = preprocess_sms1(sms)
# print(processed_sms)

In [58]:
# test_data['smsbody_2'] = test_data['smsbody_1'].apply(lambda x : preprocess_sms1(x))

In [59]:
# test_data.tail(2)

#### 4. Impute space before amount / balance decimal values and the currency symbol if there is no space.

In [60]:
# import re

# def preprocess_sms2(text):
#     # Add a space after 'Rs.' or 'INR' followed by numbers
#     pattern = r'(Rs\.|INR\.|Rs|INR)(\d)'
#     text = re.sub(pattern, r'\1 \2', text)
#     return text

In [61]:
# # Example SMS
# sms = "Your Ac No. 018/XXXXX254 has been credited by Rs.4388.On 29-MAY-2019 by Cash Info- CASH DEPOSIT . Your net available balance is Rs.4443.01"
# processed_sms = preprocess_sms(sms)
# print(processed_sms)  # Output: "NEFT txn for Rs. 50000.00 from A/c XXXX8147..."

In [62]:
# # Example SMS
# processed_sms1 = preprocess_sms(processed_sms)
# print(processed_sms1)  # Output: "NEFT txn for Rs. 50000.00 from A/c XXXX8147..."

In [63]:
# test_data['smsbody_3'] = test_data['smsbody_2'].apply(lambda x : preprocess_sms2(x))

In [64]:
# test_data.tail(2)

In [65]:
# test_data.tail(2)

In [66]:
# test_data.to_csv('temp_data_1.csv', index = False)

#### Make prediction on test data.

In [67]:
# Load the saved model
# nlp = spacy.load("path_to_model")
# path = '/home/jupyter-nikhil_singh/anduril/Rahul_Sir/Extraction_model/Final_model_8'
path = '/home/jupyter-nikhil_singh/anduril/Rahul_Sir/Extraction_model/Resume_training_of_Final_model_8'
model_name = 'custom_ner_model'

In [68]:
nlp = spacy.load(path + '/' + model_name)

In [69]:
# Function to process text and extract entities
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

In [70]:
# Apply the function to each row in the DataFrame
import time
start_time = time.time()
test_data['predicted_entities'] = test_data['final_smsbody'].apply(extract_entities)
end_time = time.time()
latency = end_time - start_time
print(f"latency : {latency}")

latency : 887.8072054386139


In [71]:
# Print the results
test_data.head(150)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,smsbody_3,smsbody_4,smsbody_5,smsbody_6,final_smsbody,predicted_entities
0,"DBGB: Ac XXXXXXXX003338 Debited with Rs.158.00,02-11-2022 17:36:11. Aval Bal Rs.5111.00 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,1a2329fdadea93a4e0f929a321fd9e54,debit-transaction,158.0,5111.0,XXXXXXXX003338,02-11-2022,"DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","[(XXXXXXXX003338, PAN), (158.00, AMOUNT), (02-11-2022, DATE), (5111.00, BALANCE)]"
1,"DBGB: Ac XXXXXXXX509006 Debited with Rs.17.70,27-06-2024 20:30:29. Aval Bal Rs.152.60 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,02d29bf975b65db3c0122bd302b6b780,debit-transaction,17.69,152.59,XXXXXXXX509006,27-06-2024,"DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70,27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70,27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","[(XXXXXXXX509006, PAN), (17.70, AMOUNT), (27-06-2024, DATE), (152.60, BALANCE)]"
2,"DBGB: Ac XXXXXXXX059544 Debited with Rs.4000.00,02-02-2023 12:08:39. Aval Bal Rs.1128.72 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,b531316c9d60e0991dd956b708a36ea3,debit-transaction,4000.0,1128.72,XXXXXXXX059544,02-02-2023,"DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00,02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00,02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00, 02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00, 02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00, 02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00, 02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX059544 Debited with Rs. 4000.00, 02-02-2023 12 : 08 : 39. Aval Bal Rs. 1128.72 CR. Helpline 18001807777","[(XXXXXXXX059544, PAN), (4000.00, AMOUNT), (02-02-2023, DATE), (1128.72, BALANCE)]"
3,"DBGB: Ac XXXXXXXX001873 Debited with Rs.2000.00,08-03-2024 14:05:33. Aval Bal Rs.1670.00 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,8c8d613712f4aa88c5e668f1fdaf84c2,debit-transaction,2000.0,1670.0,XXXXXXXX001873,08-03-2024,"DBGB : Ac XXXXXXXX001873 Debited with Rs. 2000.00,08-03-2024 14

### Post processing of predictions.

#### Getting the length of the predictions.

In [72]:
test_data['length_of_predicted_entities'] = test_data['predicted_entities'].apply(lambda x : len(x))

In [73]:
test_data['length_of_predicted_entities'].value_counts()

4     81407
3     55425
2     27467
1     10592
5      6698
0       631
6        89
7        13
8         5
9         3
11        2
Name: length_of_predicted_entities, dtype: int64

#### Flag the entites which are predicted.

In [74]:
test_data['Predicted_amount_flag'] = test_data['predicted_entities'].apply(lambda x: 1 if any('AMOUNT' in item for item in x) else 0)
test_data['Predicted_pan_flag'] = test_data['predicted_entities'].apply(lambda x: 1 if any('PAN' in item for item in x) else 0)
test_data['Predicted_date_flag'] = test_data['predicted_entities'].apply(lambda x: 1 if any('DATE' in item for item in x) else 0)
test_data['Predicted_balance_flag'] = test_data['predicted_entities'].apply(lambda x: 1 if any('BALANCE' in item for item in x) else 0)

In [75]:
test_data.head(2)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,smsbody_4,smsbody_5,smsbody_6,final_smsbody,predicted_entities,length_of_predicted_entities,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag
0,"DBGB: Ac XXXXXXXX003338 Debited with Rs.158.00,02-11-2022 17:36:11. Aval Bal Rs.5111.00 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,1a2329fdadea93a4e0f929a321fd9e54,debit-transaction,158.0,5111.0,XXXXXXXX003338,02-11-2022,"DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777",...,"DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00, 02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","[(XXXXXXXX003338, PAN), (158.00, AMOUNT), (02-11-2022, DATE), (5111.00, BALANCE)]",4,1,1,1,1
1,"DBGB: Ac XXXXXXXX509006 Debited with Rs.17.70,27-06-2024 20:30:29. Aval Bal Rs.152.60 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,02d29bf975b65db3c0122bd302b6b780,debit-transaction,17.69,152.59,XXXXXXXX509006,27-06-2024,"DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70,27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70,27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777",...,"DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70, 27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","[(XXXXXXXX509006, PAN), (17.70, AMOUNT), (27-06-2024, DATE), (152.60, BALANCE)]",4,1,1,1,1


#### Segrigate the predicted entities in columns for operation.

In [76]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182332 entries, 0 to 182331
Data columns (total 21 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   smsbody                       182332 non-null  object
 1   templatehash                  182332 non-null  object
 2   hash                          182332 non-null  object
 3   smstype                       182332 non-null  object
 4   amount                        182332 non-null  object
 5   balance                       182332 non-null  object
 6   pan                           182332 non-null  object
 7   date                          182332 non-null  object
 8   smsbody_1                     182332 non-null  object
 9   smsbody_2                     182332 non-null  object
 10  smsbody_3                     182332 non-null  object
 11  smsbody_4                     182332 non-null  object
 12  smsbody_5                     182332 non-null  object
 13 

In [77]:
# Define a function to extract 'AMOUNT', 'PAN', 'BALANCE', and 'DATE'
def extract_entities(entities):
    amount = None
    pan = None
    balance = None
    date = None
#     print(entities)
    for value, entity_type in entities:
#         print(value, entity_type)
        if entity_type == 'AMOUNT' and amount is None:  # Store the first AMOUNT found
#             print('inside amount')
            amount = value
        elif entity_type == 'PAN' and pan is None:
#             print('inside pan')
            pan = value
        elif entity_type == 'BALANCE' and balance is None:
#             print('inside balance')
            balance = value
        elif entity_type == 'DATE' and date is None:
#             print('inside date')
            date = value
    
    return pd.Series([amount, pan, balance, date])

In [78]:
# test_data[test_data['hash'] == 'ec5f9344e134ae3ea9aa897079f8b8cf']['predicted_entities'].apply( lambda x : extract_entities(x))

In [79]:
# Apply the function to the 'predicted_entity' column
test_data[['predicted_amount', 'predicted_pan', 'predicted_balance', 'predicted_date']] = test_data['predicted_entities'].apply( lambda x : extract_entities(x))

In [80]:
test_data.tail(1)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,predicted_entities,length_of_predicted_entities,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag,predicted_amount,predicted_pan,predicted_balance,predicted_date
182331,"ICICI Bank Acc XX291 debited Rs. 1,500.00 on 01-Jan-24 NFS*CASH WDL*. Avb Bal Rs. 171.74. To dispute Call 18002662 or SMS BLOCK 291 to 9215676766 .",f4b0f336ff0669fd8fdedad3f9b0ea82,c85e16dc0536d16138a6b413f143cc12,debit-transaction,1500.0,171.74,XX291,01-Jan-24,"ICICI Bank Acc XX291 debited Rs. 1,500.00 on 01-Jan-24 NFS*CASH WDL*. Avb Bal Rs. 171.74. To dispute Call 18002662 or SMS BLOCK 291 to 9215676766 .","ICICI Bank Acc XX291 debited Rs. 1,500.00 on 01-Jan-24 NFS*CASH WDL*. Avb Bal Rs. 171.74. To dispute Call 18002662 or SMS BLOCK 291 to 9215676766 .",...,"[(XX291, PAN), (1,500.00, AMOUNT), (01-Jan-24, DATE), (171.74, BALANCE)]",4,1,1,1,1,"1,500.00",XX291,171.74,01-Jan-24


In [81]:
# test_data.to_csv('unseen_test_data_acctype_bank1.csv', index = False)

#### Get the numeric part of the string captured in 'amount' and 'balance'.

In [93]:
# Define a function to extract the numeric part
def extract_numeric(text):
    # Find numeric values using regex, allowing spaces, commas (both standard and non-standard), and periods
    match = re.search(r'(?<!\d)[\d,\s，-]+(?:\.\d+)?', str(text))  # Captures numbers with spaces, commas, and decimals
    if match:
        # Clean up: replace non-standard commas and strip extra spaces
        return match.group(0).replace('，', ',').replace(' ', '')
    return None

In [99]:
# Test with problematic input
print(extract_numeric('-34478-'))  # Output: '59687.56'

-34478-


In [96]:
# Apply the function to the 'predicted_amount' column
test_data['predicted_amount'] = test_data['predicted_amount'].apply(lambda x : extract_numeric(x))

In [100]:
# Apply the function to the 'predicted_amount' column
test_data['predicted_balance'] = test_data['predicted_balance'].apply(lambda x : extract_numeric(x))

In [101]:
# test_data.to_csv('unseen_test_data_acctype_bank1.csv', index = False)

In [102]:
test_data.tail(1)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,predicted_entities,length_of_predicted_entities,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag,predicted_amount,predicted_pan,predicted_balance,predicted_date
182331,"ICICI Bank Acc XX291 debited Rs. 1,500.00 on 01-Jan-24 NFS*CASH WDL*. Avb Bal Rs. 171.74. To dispute Call 18002662 or SMS BLOCK 291 to 9215676766 .",f4b0f336ff0669fd8fdedad3f9b0ea82,c85e16dc0536d16138a6b413f143cc12,debit-transaction,1500.0,171.74,XX291,01-Jan-24,"ICICI Bank Acc XX291 debited Rs. 1,500.00 on 01-Jan-24 NFS*CASH WDL*. Avb Bal Rs. 171.74. To dispute Call 18002662 or SMS BLOCK 291 to 9215676766 .","ICICI Bank Acc XX291 debited Rs. 1,500.00 on 01-Jan-24 NFS*CASH WDL*. Avb Bal Rs. 171.74. To dispute Call 18002662 or SMS BLOCK 291 to 9215676766 .",...,"[(XX291, PAN), (1,500.00, AMOUNT), (01-Jan-24, DATE), (171.74, BALANCE)]",4,1,1,1,1,"1,500.00",XX291,171.74,01-Jan-24


#### Check the predicted values are numeric in 'amount' & 'balance'. 

In [103]:
# Function to check if a value is numeric after removing commas
def check_numeric(val):
    if val is None:  # Handle None values
        return 0
    try:
        # Remove commas from the value
        val = val.replace(',', '')
        # Try converting to numeric
        pd.to_numeric(val)
        return 1
    except ValueError:
        return 0

In [106]:
check_numeric('-234,566-')

0

In [5]:
test_data.columns

NameError: name 'test_data' is not defined

In [107]:
# Apply the function to the 'amount' column and create a new 'is_numeric_flag' column
test_data['is_predicted_amount_numeric_flag'] = test_data['predicted_amount'].apply(lambda x : check_numeric(x))

In [108]:
# Apply the function to the 'balance' column and create a new 'is_numeric_flag' column
test_data['is_Predicted_balance_numeric_flag'] = test_data['predicted_balance'].apply(lambda x : check_numeric(x))

In [109]:
test_data.tail(1)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag,predicted_amount,predicted_pan,predicted_balance,predicted_date,is_predicted_amount_numeric_flag,is_Predicted_balance_numeric_flag
182331,"ICICI Bank Acc XX291 debited Rs. 1,500.00 on 01-Jan-24 NFS*CASH WDL*. Avb Bal Rs. 171.74. To dispute Call 18002662 or SMS BLOCK 291 to 9215676766 .",f4b0f336ff0669fd8fdedad3f9b0ea82,c85e16dc0536d16138a6b413f143cc12,debit-transaction,1500.0,171.74,XX291,01-Jan-24,"ICICI Bank Acc XX291 debited Rs. 1,500.00 on 01-Jan-24 NFS*CASH WDL*. Avb Bal Rs. 171.74. To dispute Call 18002662 or SMS BLOCK 291 to 9215676766 .","ICICI Bank Acc XX291 debited Rs. 1,500.00 on 01-Jan-24 NFS*CASH WDL*. Avb Bal Rs. 171.74. To dispute Call 18002662 or SMS BLOCK 291 to 9215676766 .",...,1,1,1,1,"1,500.00",XX291,171.74,01-Jan-24,1,1


In [380]:
test_data[(test_data['Predicted_balance_flag'] == 1) & (test_data['is_Predicted_balance_numeric_flag'] == 0)]

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag,predicted_amount,predicted_pan,predicted_balance,predicted_date,is_predicted_amount_numeric_flag,is_Predicted_balance_numeric_flag
62632,राशि रु.200 TRTR/31881298786 से खाता सं 4998 में जमा की गई. कुल शेष राशि रु...27CR. उपलब्ध शेष राशि रु. 200.27(07-07-2023 12:33:01).- बैंक ऑफ़ बड़ौदा,6e31ae81ed54ac024f157f4eb1ea9321,87c8740057c980461f77be09ae42ee51,credit-transaction,200.0,200.27,4998,07-07-2023,राशि रु.200 TRTR/31881298786 से खाता सं 4998 में जमा की गई. कुल शेष राशि रु...27CR. उपलब्ध शेष राशि रु. 200.27 ( 07-07-2023 12 : 33 : 01 ) .- बैंक ऑफ़ बड़ौदा,राशि रु.200 TRTR/31881298786 से खाता सं 4998 में जमा की गई. कुल शेष राशि रु...27CR. उपलब्ध शेष राशि रु. 200.27 ( 07-07-2023 12 : 33 : 01 ) .- बैंक ऑफ़ बड़ौदा,...,0,1,1,1,None,4998,None,07-07-2023,0,0
62664,राशि रु.8000 TRTR/33211360572 से खाता सं 1649 में जमा की गई. कुल शेष राशि रु...09CR. उपलब्ध शेष राशि रु. 8000.09(17-11-2023 13:20:11).- बैंक ऑफ़ बड़ौदा,6e31ae81ed54ac024f157f4eb1ea9321,35d5862a4806c7cef506254d2238edf2,credit-transaction,8000.0,8000.09,1649,17-11-2023,राशि रु.8000 TRTR/33211360572 से खाता सं 1649 में जमा की गई. कुल शेष राशि रु...09CR. उपलब्ध शेष राशि रु. 8000.09 ( 17-11-2023 13 : 20 : 11 ) .- बैंक ऑफ़ बड़ौदा,राशि रु.8000 TRTR/33211360572 से खाता सं 1649 में जमा की गई. कुल शेष राशि रु...09CR. उपलब्ध शेष राशि रु. 8000.09 ( 17-11-2023 13 : 20 : 11 ) .- बैंक ऑफ़ बड़ौदा,...,0,1,1,1,None,1649,None,17-11-2023,0,0
62698,राशि रु.5400 TRTR/40311551876 से खाता सं 9629 में जमा की गई. कुल शेष राशि रु...33CR. उपलब्ध शेष राशि रु. 5400.33(31-01-2024 15:05:23).- बैंक ऑफ़ बड़ौदा,6e31ae81ed54ac024f157f4eb1ea9321,6ba970f66bdf340883c469ac01d4768d,credit-transaction,5400.0,5400.32,9629,31-01-2024,राशि रु.5400 TRTR/40311551876 से खाता सं 9629 में जमा की गई. कुल शेष राशि रु...33CR. उपलब्ध शेष राशि रु. 5400.33 ( 31-01-2024 15 : 05 : 23 ) .- बैंक ऑफ़ बड़ौदा,राशि रु.5400 TRTR/40311551876 से खाता सं 9629 में जमा की गई. कुल शेष राशि रु...33CR. उपलब्ध शेष राशि रु. 5400.33 ( 31-01-2024 15 : 05 : 23 ) .- बैंक ऑफ़ बड़ौदा,...,0,1,1,1,None,9629,None,31-01-2024,0,0
63167,राशि रु.8000 TRTR/41841607267 से खाता सं 7813 में जमा की गई. कुल शेष राशि रु...33CR. उपलब्ध शेष राशि रु. 10000.33(02-07-2024 16:50:20).- बैंक ऑफ़ बड़ौदा,6e31ae81ed54ac024f157f4eb1ea9321,12d0749dc007a5fc70a4c20db41066be,credit-transaction,8000.0,10000.33,7813,02-07-2024,राशि रु.8000 TRTR/41841607267 से खाता सं 7813 में जमा की गई. कुल शेष राशि रु...33CR. उपलब्ध शेष राशि रु. 10000.33 ( 02-07-2024 16 : 50 : 20 ) .- बैंक ऑफ़ बड़ौदा,राशि रु.8000 TRTR/41841607267 से खाता सं 7813 में जमा की गई. कुल शेष राशि रु...33CR. उपलब्ध शेष राशि रु. 10000.33 ( 02-07-2024 16 : 50 : 20 ) .- बैंक ऑफ़ बड़ौदा,...,0,1,1,1,None,7813,None,02-07-2024,0,0
63497,राशि रु.10000 TRTR/33181341815 से खाता सं 1649 में जमा की गई. कुल शेष राशि रु...09CR. उपलब्ध शेष राशि रु. 10000.09(14-11-2023 13:00:38).- बैंक ऑफ़ बड़ौदा,6e31ae81ed54ac024f157f4eb1ea9321,5d0a688e00a40ec6dd4311e1b95669c1,credit-transaction,10000.0,10000.09,1649,14-11-2023,राशि रु.10000 TRTR/33181341815 से खाता सं 1649 में जमा की गई. कुल शेष राशि रु...09CR. उपलब्ध शेष राशि रु. 10000.09 ( 14-11-2023 13 : 00 : 38 ) .- बैंक ऑफ़ बड़ौदा,राशि रु.10000 TRTR/33181341815 से खाता सं 1649 में जमा की गई. कुल शेष राशि रु...09CR. उपलब्ध शेष राशि रु. 10000.09 ( 14-11-2023 13 : 00 : 38 ) .- बैंक ऑफ़ बड़ौदा,...,0,1,1,1,None,1649,None,14-11-2023,0,0
63579,राशि रु.3000 TRTR/42791850985 से खाता सं 7171 में जमा की गई. कुल शेष राशि रु...5CR. उपलब्ध शेष राशि रु. 3000.5(05-10-2024 18:53:01).- बैंक ऑफ़ बड़ौदा,6e31ae81ed54ac024f157f4eb1ea9321,175cde0381c2bd67b63360fbeeda8dbc,credit-transaction,3000.0,3000.0,7171,05-10-2024,राशि रु.3000 TRTR/42791850985 से खाता सं 7171 में जमा की गई. कुल शेष राशि रु...5CR. उपलब्ध शेष राशि रु. 3000.5 ( 05-10-2024 18 : 53 : 01 ) .- बैंक ऑफ़ बड़ौदा,राशि रु.3000 TRTR/42791850985 से खाता सं 7171 में जमा की गई. कुल शेष राशि रु...5CR. उपलब्ध शेष राशि रु. 3000.5 ( 05-10-2024

#### Write the data generated above into S3 to analyse the templates containing errors.

In [110]:
# test_data.to_csv('prediction_done_on_test_data_2.csv', index = False)

#### Get those templates where 'amount' and 'balance' got predicted by the model but they not numeric values.

In [111]:
test_data[(test_data['Predicted_amount_flag'] == 1) & (test_data['is_predicted_amount_numeric_flag'] == 0)][['templatehash', 'hash', 'smsbody', 'final_smsbody',
'predicted_entities',
'length_of_predicted_entities',
'Predicted_amount_flag',  
'Predicted_pan_flag',   
'Predicted_date_flag',
'Predicted_balance_flag']].shape

(12, 10)

In [115]:
test_data[(test_data['Predicted_amount_flag'] == 1) & (test_data['is_predicted_amount_numeric_flag'] == 0)].groupby(['templatehash'])['hash'].nunique()

templatehash
0d4664cd685d8114b56713711e139943     1
51594b3e12f22c2b2e69963c462a2c94    11
Name: hash, dtype: int64

In [ ]:
test_data[(test_data['Predicted_balance_flag'] == 1) & (test_data['is_Predicted_balance_numeric_flag'] == 0)][['templatehash', 'hash', 'smsbody', 'final_smsbody',
'predicted_entities',
'length_of_predicted_entities',
'Predicted_amount_flag',  
'Predicted_pan_flag',   
'Predicted_date_flag',
'Predicted_balance_flag']].shape

In [116]:
test_data[(test_data['Predicted_balance_flag'] == 1) & (test_data['is_Predicted_balance_numeric_flag'] == 0)].groupby(['templatehash'])['hash'].nunique()

templatehash
6e31ae81ed54ac024f157f4eb1ea9321    7
c40528cbe9cc2aa5d8b3a21c71067b78    2
Name: hash, dtype: int64

### Calculate extraction report/metric with respect to extracted data.

#### 1. Extraction

In [359]:
# Assuming the counts are already calculated
actual_amt_cnt = test_data[~test_data['amount'].isin([-1, '-1', '-1.0', -1.0])]['hash'].nunique()
predicted_amt_cnt = test_data[~test_data['amount'].isin([-1, '-1', '-1.0', -1.0])]['Predicted_amount_flag'].sum()

In [360]:
actual_pan_cnt = test_data[~test_data['pan'].isin([-1, '-1', '-1.0', -1.0])]['hash'].nunique()
predicted_pan_cnt = test_data[~test_data['pan'].isin([-1, '-1', '-1.0', -1.0])]['Predicted_pan_flag'].sum()

In [361]:
actual_date_cnt = test_data[~test_data['date'].isin([-1, '-1', '-1.0', -1.0])]['hash'].nunique()
predicted_date_cnt = test_data[~test_data['date'].isin([-1, '-1', '-1.0', -1.0])]['Predicted_date_flag'].sum()

In [362]:
actual_balance_cnt = test_data[~test_data['balance'].isin([-1, '-1', '-1.0', -1.0])]['hash'].nunique()
predicted_balance_cnt = test_data[~test_data['balance'].isin([-1, '-1', '-1.0', -1.0])]['Predicted_balance_flag'].sum()

In [363]:
actual_balance_cnt

101947

In [364]:
# Create a dictionary for actual and predicted counts
data = {
    'Actual Count': [actual_amt_cnt, actual_pan_cnt, actual_date_cnt, actual_balance_cnt],
    'Predicted Count': [predicted_amt_cnt, predicted_pan_cnt, predicted_date_cnt, predicted_balance_cnt]
}

In [365]:
# Create the DataFrame with appropriate labels
metrics_df = pd.DataFrame(data, index=['Amount', 'PAN', 'Date', 'Balance'])

In [366]:
# Display the matrix form of the results
# Calculate accuracy percentage
metrics_df['Unextracted Count'] = (metrics_df['Actual Count'] - metrics_df['Predicted Count'])
metrics_df['Extraction_%'] = round((metrics_df['Predicted Count'] / metrics_df['Actual Count']) * 100, 2)

In [367]:
# Display the updated metrics DataFrame
metrics_df

,Actual Count,Predicted Count,Unextracted Count,Extraction_%
Amount,182261,176447,5814,96.81
PAN,159037,149684,9353,94.12
Date,149127,143087,6040,95.95
Balance,101947,90914,11033,89.18


##### Get the missing / unextracted templates to anlayse.

##### Amt.

In [368]:
test_data[(~test_data['amount'].isin([-1, '-1', '-1.0', -1.0])) &
                        (test_data['Predicted_amount_flag'] == 0)][['templatehash', 'hash', 'smsbody', 'final_smsbody', 'amount',
            'predicted_entities',
            'predicted_amount',
            'Predicted_amount_flag', 'is_predicted_amount_numeric_flag']].shape

(5814, 9)

In [249]:
missing_amt = test_data[(~test_data['amount'].isin([-1, '-1', '-1.0', -1.0])) &
                        (test_data['Predicted_amount_flag'] == 0)][['templatehash']]

In [250]:
missing_amt

,templatehash
3502,87ef3be73ec13ce91ddd7ea26f1cf018
3503,87ef3be73ec13ce91ddd7ea26f1cf018
3504,87ef3be73ec13ce91ddd7ea26f1cf018
3505,87ef3be73ec13ce91ddd7ea26f1cf018
3506,87ef3be73ec13ce91ddd7ea26f1cf018
...,...
177763,c40528cbe9cc2aa5d8b3a21c71067b78
177767,c40528cbe9cc2aa5d8b3a21c71067b78
177769,c40528cbe9cc2aa5d8b3a21c71067b78
177770,c40528cbe9cc2aa5d8b3a21c71067b78


In [251]:
missing_amt['missing'] = 'Amt'

In [252]:
missing_amt = missing_amt.drop_duplicates(subset=['templatehash'], ignore_index=True)

In [253]:
missing_amt.shape

(39, 2)

In [254]:
missing_amt.head()

,templatehash,missing
0,87ef3be73ec13ce91ddd7ea26f1cf018,Amt
1,65f3d395b13dd8f37838b0efe8da9dd8,Amt
2,a1e293ace83caef893a7d16fcac194b2,Amt
3,0d4664cd685d8114b56713711e139943,Amt
4,8fc7cc3ed91e141e34d85b8464e5b1cb,Amt


In [255]:
# missing_amt[missing_amt['templatehash'] == '1abbdfedd0cfec9b267e4c791a6eb7a9']

##### Balance.

In [369]:
test_data[(~test_data['balance'].isin([-1, '-1', '-1.0', -1.0])) &
          (test_data['Predicted_balance_flag'] == 0)][['templatehash', 'hash', 'smsbody', 'final_smsbody', 'balance',
            'predicted_entities',
            'predicted_balance',
            'Predicted_balance_flag', 'is_Predicted_balance_numeric_flag']].shape

(11033, 9)

In [259]:
missing_bal = test_data[(~test_data['balance'].isin([-1, '-1', '-1.0', -1.0])) &
                             (test_data['Predicted_balance_flag'] == 0)][['templatehash']]

In [260]:
missing_bal['missing'] = 'Bal'

In [261]:
missing_bal = missing_bal.drop_duplicates(subset=['templatehash'], ignore_index=True)

In [262]:
missing_bal.shape

(65, 2)

In [263]:
missing_bal.head()

,templatehash,missing
0,87ef3be73ec13ce91ddd7ea26f1cf018,Bal
1,bc781bfc086df3837e5d976f6c600b79,Bal
2,c6e7a7e27fa67190ef1784157d665d72,Bal
3,eff7090599a756dedd76bab106db75ad,Bal
4,3bcec7307f3e87edf842dfaa307c9354,Bal


##### Pan.

In [273]:
test_data[(~test_data['pan'].isin([-1, '-1', '-1.0', -1.0])) &
                             (test_data['Predicted_pan_flag'] == 0)].shape

(9353, 27)

In [264]:
missing_pan = test_data[(~test_data['pan'].isin([-1, '-1', '-1.0', -1.0])) &
                             (test_data['Predicted_pan_flag'] == 0)][['templatehash']]

In [265]:
missing_pan['missing'] = 'Pan'

In [266]:
missing_pan = missing_pan.drop_duplicates(subset=['templatehash'], ignore_index=True)

In [267]:
missing_pan.shape

(25, 2)

In [268]:
missing_pan.head()

,templatehash,missing
0,57982c486804d4068efa86fca5991bae,Pan
1,eff7090599a756dedd76bab106db75ad,Pan
2,868f7aaf8b632326fc86a79d4a45b351,Pan
3,3c601a1c3609b188c765d4bd9aaf690a,Pan
4,f42e98998939e2f7e8b2fc7373514e9d,Pan


##### Date.

In [274]:
test_data[(~test_data['date'].isin([-1, '-1', '-1.0', -1.0])) &
                             (test_data['Predicted_date_flag'] == 0)].shape

(6040, 27)

In [275]:
missing_date = test_data[(~test_data['date'].isin([-1, '-1', '-1.0', -1.0])) &
                             (test_data['Predicted_date_flag'] == 0)][['templatehash']]

In [276]:
missing_date['missing'] = 'date'

In [277]:
missing_date = missing_date.drop_duplicates(subset=['templatehash'], ignore_index=True)

In [278]:
missing_date.shape

(45, 2)

In [279]:
missing_date.head()

,templatehash,missing
0,9f3e5a575d550be631cdcf29f920c769,date
1,94f9c43419738ab4a47fabcb7ee30eae,date
2,6ff735f39f99f2496c4740b604072ebe,date
3,2e71e66574aa77571114a3df2bc95853,date
4,20ead8b77d71e14ce29b2e011f1a1d54,date


##### Combining all the dataframes to create a collated erroneous templates data to analyse.

In [280]:
# Assuming you have a list of dataframes
dataframes = [missing_amt, missing_bal, missing_pan, missing_date]  # Replace with your actual dataframe variables

In [281]:
# Concatenate all dataframes
combined_missing_df = pd.concat(dataframes)

In [282]:
combined_missing_df.shape

(174, 2)

In [381]:
# combined_missing_df.to_csv('combined_missing_extraction_df.csv', index = False)

In [284]:
combined_missing_df.groupby('templatehash')['missing']

In [285]:
# Group by 'templates' and aggregate 'missing' values
missing_templates_df = combined_missing_df.groupby('templatehash')['missing'].apply(lambda x: ', '.join(x)).reset_index()

In [286]:
# Display the result
missing_templates_df

,templatehash,missing
0,02597a5b63c22eea0cc716f7ba0bb820,Amt
1,06f77e41d0d4dfc50956f870cfd28f7c,Pan
2,08769065948716ce74696f789b66a332,"Amt, date"
3,09dc221114aad241d5ddc945b8ec1ded,Bal
4,0a3431132fffed9366ad86b0d8e505a7,Pan
...,...,...
131,f3005925aacaa6305d6365ea64005b14,"Amt, Bal"
132,f42e98998939e2f7e8b2fc7373514e9d,Pan
133,f78551e95fab2b3827d5fae6781549cc,Pan
134,fc88e3e496158621bf9233c25f62d5b1,Bal


In [382]:
# missing_templates_df.to_excel('unextracted_templates_with missing_reason.xlsx', index = False)

### Calculate accuracy report/metric with respect to extracted data.

#### Calculate regression metrics for Amount.

In [288]:
test_data.columns

Index(['smsbody', 'templatehash', 'hash', 'smstype', 'amount', 'balance',
       'pan', 'date', 'smsbody_1', 'smsbody_2', 'smsbody_3', 'smsbody_4',
       'smsbody_5', 'smsbody_6', 'final_smsbody', 'predicted_entities',
       'length_of_predicted_entities', 'Predicted_amount_flag',
       'Predicted_pan_flag', 'Predicted_date_flag', 'Predicted_balance_flag',
       'predicted_amount', 'predicted_pan', 'predicted_balance',
       'predicted_date', 'is_predicted_amount_numeric_flag',
       'is_Predicted_balance_numeric_flag'],
      dtype='object')

In [289]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182332 entries, 0 to 182331
Data columns (total 27 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   smsbody                            182332 non-null  object
 1   templatehash                       182332 non-null  object
 2   hash                               182332 non-null  object
 3   smstype                            182332 non-null  object
 4   amount                             182332 non-null  object
 5   balance                            182332 non-null  object
 6   pan                                182332 non-null  object
 7   date                               182332 non-null  object
 8   smsbody_1                          182332 non-null  object
 9   smsbody_2                          182332 non-null  object
 10  smsbody_3                          182332 non-null  object
 11  smsbody_4                          182332 non-null  

In [290]:
amt_df = test_data[
                  (~test_data['amount'].isin([-1, -1.0, -2, -2.0, '-1', '-1.0', '-2', '-2.0'])) &
                  (test_data['is_predicted_amount_numeric_flag'] == 1)
                  ]

In [291]:
amt_df.shape

(176419, 27)

In [292]:
amt_df['amount'].min()

'0.0'

In [293]:
amt_df.head(2)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag,predicted_amount,predicted_pan,predicted_balance,predicted_date,is_predicted_amount_numeric_flag,is_Predicted_balance_numeric_flag
0,"DBGB: Ac XXXXXXXX003338 Debited with Rs.158.00,02-11-2022 17:36:11. Aval Bal Rs.5111.00 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,1a2329fdadea93a4e0f929a321fd9e54,debit-transaction,158.0,5111.0,XXXXXXXX003338,02-11-2022,"DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777",...,1,1,1,1,158.00,XXXXXXXX003338,5111.00,02-11-2022,1,1
1,"DBGB: Ac XXXXXXXX509006 Debited with Rs.17.70,27-06-2024 20:30:29. Aval Bal Rs.152.60 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,02d29bf975b65db3c0122bd302b6b780,debit-transaction,17.69,152.59,XXXXXXXX509006,27-06-2024,"DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70,27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70,27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777",...,1,1,1,1,17.70,XXXXXXXX509006,152.60,27-06-2024,1,1


In [294]:
amt_df[amt_df['amount'].isin([-2, -2.0, '-2', '-2.0'])]

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag,predicted_amount,predicted_pan,predicted_balance,predicted_date,is_predicted_amount_numeric_flag,is_Predicted_balance_numeric_flag


In [371]:
# Suppress any warnings.
import warnings
warnings.filterwarnings('ignore')

# Convert all values to strings, remove commas, then convert to float, and handle missing values
amt_df['amount_cleaned'] = amt_df['amount'].astype(str).str.replace(',', '').replace('None', np.nan).astype(float)
amt_df['predicted_amount_cleaned'] = amt_df['predicted_amount'].astype(str).str.replace(',', '').replace('None', np.nan).astype(float)

# mean_amount = amt_df['amount_cleaned'].mean()
# mean_prdicted_amount = amt_df['predicted_amount_cleaned'].mean()

# print("Mean of Actual amount:", mean_amount)
# print("Mean of Predicted amount:", mean_prdicted_amount)

pd.options.display.float_format = '{:.2f}'.format
print(amt_df[['amount_cleaned', 'predicted_amount_cleaned']].describe(percentiles = [0.01, 0.02, 0.03, 0.04, 0.05, 0.25, 0.50, 0.75, 0.95, 0.96, 0.97, 0.98, 0.99]))

# # Calculate Mean Squared Error, skipping rows with NaN values
# mse = np.nanmean((amt_df['amount_cleaned'] - amt_df['predicted_amount_cleaned']) ** 2)
# print("Mean Squared Error of Amount:",round(mse, 2))

# Calculate Mean Squared Error, skipping rows with NaN values
mae = np.mean(np.abs(amt_df['amount_cleaned'] - amt_df['predicted_amount_cleaned']))
print("Mean Absolute Error of Amount:", round(mae, 2))

       amount_cleaned  predicted_amount_cleaned
count       176419.00                 176419.00
mean         14272.93                  14272.32
std          91059.02                  91059.10
min              0.00                      0.00
1%               1.00                      1.00
2%               1.00                      1.00
3%               2.64                      3.00
4%               7.99                      7.00
5%              10.00                     10.00
25%            185.00                    183.00
50%           1020.00                   1020.00
75%           7050.00                   7050.00
95%          50000.00                  50000.00
96%          60000.00                  60000.00
97%          85000.00                  85000.00
98%         110000.00                 110000.00
99%         205000.00                 205000.00
max        8290623.95                8290623.95
Mean Absolute Error of Amount: 0.73


##### Get the templates for which wrong prediction has been made.

##### For Amount

In [2]:
# test_data[(~test_data['amount'].isin([-1, -1.0, -2, -2.0, '-1', '-1.0', '-2', '-2.0'])) &
#           (test_data['Predicted_amount_flag'] == 1) &
#           (test_data['is_predicted_amount_numeric_flag'] == 0)
#          ][['templatehash', 'hash', 'smsbody', 'final_smsbody', 'amount',
#             'predicted_entities',
#             'predicted_balance',
#             'length_of_predicted_entities',
#             'Predicted_amount_flag', 'is_predicted_amount_numeric_flag']].groupby(['templatehash'])['hash'].nunique()

#### Calculate regression metrics for Balance.

In [297]:
test_data.columns

Index(['smsbody', 'templatehash', 'hash', 'smstype', 'amount', 'balance',
       'pan', 'date', 'smsbody_1', 'smsbody_2', 'smsbody_3', 'smsbody_4',
       'smsbody_5', 'smsbody_6', 'final_smsbody', 'predicted_entities',
       'length_of_predicted_entities', 'Predicted_amount_flag',
       'Predicted_pan_flag', 'Predicted_date_flag', 'Predicted_balance_flag',
       'predicted_amount', 'predicted_pan', 'predicted_balance',
       'predicted_date', 'is_predicted_amount_numeric_flag',
       'is_Predicted_balance_numeric_flag'],
      dtype='object')

In [298]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182332 entries, 0 to 182331
Data columns (total 27 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   smsbody                            182332 non-null  object
 1   templatehash                       182332 non-null  object
 2   hash                               182332 non-null  object
 3   smstype                            182332 non-null  object
 4   amount                             182332 non-null  object
 5   balance                            182332 non-null  object
 6   pan                                182332 non-null  object
 7   date                               182332 non-null  object
 8   smsbody_1                          182332 non-null  object
 9   smsbody_2                          182332 non-null  object
 10  smsbody_3                          182332 non-null  object
 11  smsbody_4                          182332 non-null  

In [299]:
balance_df = test_data[
                     (~test_data['balance'].isin([-1, -1.0, -2, -2.0, '-1', '-1.0', '-2', '-2.0'])) &
                     (test_data['is_Predicted_balance_numeric_flag'] == 1)
                     ]

In [300]:
balance_df.shape

(90867, 27)

In [301]:
balance_df['balance'].min()

'-1005000.0'

In [302]:
balance_df[balance_df['balance'].isin([-2, -2.0, '-2', '-2.0'])]

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag,predicted_amount,predicted_pan,predicted_balance,predicted_date,is_predicted_amount_numeric_flag,is_Predicted_balance_numeric_flag


In [372]:
# Suppress any warnings.
import warnings
warnings.filterwarnings('ignore')

# Convert all values to strings, remove commas, then convert to float, and handle missing values
balance_df['balance_cleaned'] = balance_df['balance'].astype(str).str.replace(',', '').replace('None', np.nan).astype(float)
balance_df['predicted_balance_cleaned'] = balance_df['predicted_balance'].astype(str).str.replace(',', '').replace('None', np.nan).astype(float)

pd.options.display.float_format = '{:.2f}'.format
print(balance_df[['balance_cleaned', 'predicted_balance_cleaned']].describe(percentiles = [0.01, 0.02, 0.03, 0.04, 0.05, 0.25, 0.50, 0.75, 0.95, 0.96, 0.97, 0.98, 0.99]))

# Calculate Mean Squared Error, skipping rows with NaN values
mae = np.mean(np.abs(balance_df['balance_cleaned'] - balance_df['predicted_balance_cleaned']))
print("Mean Absolute Error of Balance:", round(mae, 2))

       balance_cleaned  predicted_balance_cleaned
count         90867.00                   90867.00
mean          51232.41            132838429686.47
std          290255.13          39888475504148.61
min       -24578817.89           -227316092245.00
1%                0.00                       0.00
2%                1.15                       1.00
3%                6.41                       5.68
4%               16.17                      15.11
5%               30.86                      29.50
25%            1037.37                    1030.55
50%            6697.01                    6654.88
75%           27026.96                   27009.93
95%          203437.03                  206890.98
96%          258973.12                  265927.77
97%          338908.60                  348961.66
98%          475765.42                  490152.28
99%          824230.39                  942850.16
max         8504894.47       12024043019470588.00
Mean Absolute Error of Balance: 132855813089.95


In [304]:
balance_df[(balance_df['predicted_balance_cleaned'] > 5202308186)]['templatehash'].unique()

array(['9a2915937ca0fca9c13730f3caf247f9',
       'f10d170187479be3493ff0586a2be96c',
       'ac3cb5b816bce0e4934bf1908eb87b8c'], dtype=object)

##### For Balance

In [4]:
# test_data[(~test_data['balance'].isin([-1, -1.0, -2, -2.0, '-1', '-1.0', '-2', '-2.0'])) &
#           (test_data['Predicted_balance_flag'] == 1) &
#           (test_data['is_Predicted_balance_numeric_flag'] == 0)
#          ][['templatehash', 'hash', 'smsbody', 'final_smsbody', 'balance',
#             'predicted_entities',
#             'predicted_balance',
#             'length_of_predicted_entities',
#             'Predicted_balance_flag', 'is_Predicted_balance_numeric_flag']].groupby(['templatehash'])['hash'].nunique()

#### Calculate accuracy metric for date.

In [306]:
test_data.head(2)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag,predicted_amount,predicted_pan,predicted_balance,predicted_date,is_predicted_amount_numeric_flag,is_Predicted_balance_numeric_flag
0,"DBGB: Ac XXXXXXXX003338 Debited with Rs.158.00,02-11-2022 17:36:11. Aval Bal Rs.5111.00 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,1a2329fdadea93a4e0f929a321fd9e54,debit-transaction,158.0,5111.0,XXXXXXXX003338,02-11-2022,"DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777",...,1,1,1,1,158.00,XXXXXXXX003338,5111.00,02-11-2022,1,1
1,"DBGB: Ac XXXXXXXX509006 Debited with Rs.17.70,27-06-2024 20:30:29. Aval Bal Rs.152.60 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,02d29bf975b65db3c0122bd302b6b780,debit-transaction,17.69,152.59,XXXXXXXX509006,27-06-2024,"DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70,27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX509006 Debited with Rs. 17.70,27-06-2024 20 : 30 : 29. Aval Bal Rs. 152.60 CR. Helpline 18001807777",...,1,1,1,1,17.70,XXXXXXXX509006,152.60,27-06-2024,1,1


In [307]:
date_df = test_data[(~test_data['date'].isin([-1, -1.0, '-1', '-1.0'])) & (test_data['Predicted_date_flag'] == 1)]

In [308]:
date_df.head(1)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag,predicted_amount,predicted_pan,predicted_balance,predicted_date,is_predicted_amount_numeric_flag,is_Predicted_balance_numeric_flag
0,"DBGB: Ac XXXXXXXX003338 Debited with Rs.158.00,02-11-2022 17:36:11. Aval Bal Rs.5111.00 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,1a2329fdadea93a4e0f929a321fd9e54,debit-transaction,158.0,5111.0,XXXXXXXX003338,02-11-2022,"DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777",...,1,1,1,1,158.00,XXXXXXXX003338,5111.00,02-11-2022,1,1


In [309]:
date_df.shape

(143087, 27)

In [310]:
cnt_unique_date = date_df[(date_df['date'] == date_df['predicted_date'])]['hash'].nunique()

In [311]:
cnt_unique_date

134913

In [312]:
date_df_shape = date_df.shape[0]

In [313]:
date_df_shape

143087

In [314]:
print(f'Accuracy metric for predicted date : {cnt_unique_date/date_df_shape*100:.2f}')

Accuracy metric for predicted date : 94.29


##### Get the templates for which the wrong / mismatched prediction has been made.

##### For Date

In [315]:
date_df[(date_df['date'] != date_df['predicted_date'])].shape

(8174, 27)

In [316]:
inaccurate_date=pd.DataFrame(date_df[(date_df['date'] != date_df['predicted_date'])].groupby(['templatehash'])['hash'].nunique()

templatehash
1b69b8a864fe51c352e5933988f2e1de     47
268d50601d6927ea0420bee0166a68be     17
3331cbed8baa995fcd43a3e1f8d6e48e    890
3692999a920242a6fa648a21f4505ffb     15
3c7d518a746362597ae386ca5e73b09f     37
3ed7ad62854743042bff9a22d9626828     23
404f60a0bc6c93a7154f40fafb70662a     92
42d6d0337f9bfef57dab4fd667947fc8     72
4d90ae3fa48e3b38226dfe5d92571831     43
534be7405affa3553bffd1ca3d1d164c    831
6ff735f39f99f2496c4740b604072ebe      1
8c54434ccd0225b39f790294a1895a84      7
96b1f2aebee54ac4c32a8d2fe5ab2825    984
9beea698a379242a24c5e6ffcfccee68    961
a37688252b205e4c55fcf57f777c7e5f    995
ad12af9e66f365b7ab8fc29bc962616b    996
afae0d16604b91aeb0965f86a9d69704      2
b81997d2feac3444a21f47863daefaf6     21
c40528cbe9cc2aa5d8b3a21c71067b78     57
c6e7a7e27fa67190ef1784157d665d72    956
d67847e1a577b14c5e9b03da9202138d    986
e31573703233de64bd5c1b33d707f9fa      1
eba8feb62ab02b21c5e39e1307203e6b     94
ef95df63443b38223d0215751bb92060     45
f3005925aacaa6305d6365ea640

In [317]:
date_df[(date_df['templatehash'] == 'afae0d16604b91aeb0965f86a9d69704')][['date', 'predicted_date']]

,date,predicted_date
145141,26 NOV,26 NOV 08
145151,25 NOV,25 NOV 20


#### Calculate accuracy metric for pan.

In [318]:
test_data.head(1)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag,predicted_amount,predicted_pan,predicted_balance,predicted_date,is_predicted_amount_numeric_flag,is_Predicted_balance_numeric_flag
0,"DBGB: Ac XXXXXXXX003338 Debited with Rs.158.00,02-11-2022 17:36:11. Aval Bal Rs.5111.00 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,1a2329fdadea93a4e0f929a321fd9e54,debit-transaction,158.0,5111.0,XXXXXXXX003338,02-11-2022,"DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777",...,1,1,1,1,158.00,XXXXXXXX003338,5111.00,02-11-2022,1,1


In [319]:
pan_df = test_data[(~test_data['pan'].isin([-1, -1.0, '-1', '-1.0'])) & (test_data['Predicted_pan_flag'] == 1)]

In [320]:
pan_df.head(1)

,smsbody,templatehash,hash,smstype,amount,balance,pan,date,smsbody_1,smsbody_2,...,Predicted_amount_flag,Predicted_pan_flag,Predicted_date_flag,Predicted_balance_flag,predicted_amount,predicted_pan,predicted_balance,predicted_date,is_predicted_amount_numeric_flag,is_Predicted_balance_numeric_flag
0,"DBGB: Ac XXXXXXXX003338 Debited with Rs.158.00,02-11-2022 17:36:11. Aval Bal Rs.5111.00 CR. Helpline 18001807777",4bb988f625ed5311465b87a3b42984cd,1a2329fdadea93a4e0f929a321fd9e54,debit-transaction,158.0,5111.0,XXXXXXXX003338,02-11-2022,"DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777","DBGB : Ac XXXXXXXX003338 Debited with Rs. 158.00,02-11-2022 17 : 36 : 11. Aval Bal Rs. 5111.00 CR. Helpline 18001807777",...,1,1,1,1,158.00,XXXXXXXX003338,5111.00,02-11-2022,1,1


In [321]:
pan_df.shape

(149684, 27)

In [322]:
cnt_unique_pan = pan_df[(pan_df['pan'] == pan_df['predicted_pan'])]['hash'].nunique()

In [323]:
cnt_unique_pan

144895

In [324]:
pan_df_shape = pan_df.shape[0]

In [325]:
pan_df_shape

149684

In [326]:
print(f'Accuracy metric for predicted PAN : {cnt_unique_pan/pan_df_shape*100:.2f}')

Accuracy metric for predicted PAN : 96.80


##### For Pan

In [327]:
date_df[(date_df['pan'] != date_df['predicted_pan'])].shape

(17118, 27)

In [328]:
inaccurate_pan=pd.DataFrame(date_df[(date_df['pan'] != date_df['predicted_pan'])].groupby(['templatehash'])['hash'].nunique()).reset_index()

templatehash
01cea2e18c8c7c5c2e4cba1420949afb      2
06f77e41d0d4dfc50956f870cfd28f7c    448
0a3431132fffed9366ad86b0d8e505a7     91
0e7f87b5e9c0dbcdcb5cdd8fe892bebf    254
14fe676de66cefae7e4ebb2894d139c0      4
172d77af05f616a270667b4b629581ab    771
1891911a7f106269d66696391eebdc24    950
1a95934ad14a7c18e6169eb0c19b866b      6
1c575f97d2bf7684e8dd349aedea2bb7    923
2e71e66574aa77571114a3df2bc95853     78
36bfe2067ec7a280e9ee9005c5c9db4d     81
3c601a1c3609b188c765d4bd9aaf690a    640
45fa4b8e5a0d11ec3b74aaaac8e2bbfe     62
46d936960adb62d1a38145054080c9d1    971
480e4ad5973b59cf3d6d6eb93f6e9592     76
51594b3e12f22c2b2e69963c462a2c94     11
53db68275a64f59dc649c2c49da31d7a     24
576eb651a273caf7112d1e9b1f7260fe    983
5849c113c495bc4b8399e6e49827b25b     16
60f3740d0166d947cefd256727f5e066      5
63b3b84dfa9bf16f0a0fcdffdb858224      5
668931d929042fc8aab7ce3bfec479ff     24
6f306a61f6a8c9317e98300bc97bfb9a     10
753e8e012532a7a7e6a23f8bbabaca72    198
8513e9ad6205bbf20b9c1c24265

In [329]:
date_df[(date_df['templatehash'] == '01cea2e18c8c7c5c2e4cba1420949afb')][['pan', 'predicted_pan']]

,pan,predicted_pan
177778,0701,XX0701
177779,0701,XX0701


#### Improvement
#### 3. Identify templates in which actually it is not present but it got predicted by the model.

In [221]:
templates_missing_pan = list(test_data[test_data['pan'].isin([-1, '-1', '-1.0', -1.0]) & (test_data['Predicted_pan_flag'] == 1)]['templatehash'].unique())

In [222]:
len(templates_missing_pan)

10

In [223]:
templates_missing_date = list(test_data[test_data['date'].isin([-1, '-1', '-1.0', -1.0]) &  (test_data['Predicted_date_flag'] == 1)]['templatehash'].unique())

In [224]:
len(templates_missing_date)

8

In [225]:
templates_missing_date

['3f3701c543d4f793fe3f15a6d7de5317',
 '49a92a20081b85a508b49b573881dfac',
 '741668aab36886592871cb985104b63b',
 '60c413cd5d488638ee07c516b64bbc2a',
 '681412b33f018e458b61647e7e9be8c3',
 '031913f4b646ab9f8374ea56a64c566c',
 '696b43b97e5aae76b3db04776a987157',
 '913c764e83caa7927417b9ab0a35ef4a']

In [226]:
templates_missing_amount = list(test_data[test_data['amount'].isin([-1, '-1', '-1.0', -1.0]) & (test_data['Predicted_amount_flag'] == 1)]['templatehash'].unique())

In [227]:
len(templates_missing_amount)

2

In [228]:
templates_missing_balance = list(test_data[test_data['balance'].isin([-1, '-1', '-1.0', -1.0]) &  (test_data['Predicted_balance_flag'] == 1)]['templatehash'].unique())

In [229]:
len(templates_missing_balance)

22

In [230]:
templates_missing_balance

['e01fe3c52ab21d01dcf7953ce0d24785',
 'a37688252b205e4c55fcf57f777c7e5f',
 '06f77e41d0d4dfc50956f870cfd28f7c',
 '49a92a20081b85a508b49b573881dfac',
 'd7ec08bcb6f27e44dabc731ff0864f6f',
 '1d1f7d72912d60c6a48d5a47fa34b068',
 '480e4ad5973b59cf3d6d6eb93f6e9592',
 '780270992b38a288a939f804ec085da6',
 '6ff735f39f99f2496c4740b604072ebe',
 '8513e9ad6205bbf20b9c1c242654dbe0',
 '741668aab36886592871cb985104b63b',
 '955d4e33183f4a77d576e7fe7de4625d',
 '7fe3847f2d446b87ff420c62348cfda3',
 '1be8de1e3e5dc4e0f2925dba3b3ddbf7',
 '681412b33f018e458b61647e7e9be8c3',
 '1a95934ad14a7c18e6169eb0c19b866b',
 '696b43b97e5aae76b3db04776a987157',
 '590ba0c0e71392674460b0a85eed5d82',
 '913c764e83caa7927417b9ab0a35ef4a',
 'c29f5649e513714eecff76fc31f7c9b2',
 'df1c5a7fbeb8ce5cd01d9a4d307e4b35',
 'bcb36d37788b55f744b42624fc1ed159']

In [231]:
# unique templates to correct.
unique_templates_to_correct = set(templates_missing_pan + templates_missing_date + templates_missing_amount + templates_missing_balance)

In [232]:
len(unique_templates_to_correct)

34

In [ ]:
!pip freeze > requirements.txt